### 1、处理流水数据（加五个特征）'avg_discount', 'avg_paid', 'avg_order', 'avg_balance', 'avg_advance'
##### 以下是相关的训练和测试文件

###### 训练1/train_both_his2.txt
###### 训练1/train_combined_with_merchant_info_with_store_count.txt
###### 测试1/test_both_his2.txt
###### 测试1/test_combined_with_merchant_info_with_store_count.txt

#### 1.1 查看数据是否充足

In [22]:
import pandas as pd

# 文件路径
train_both_path = '训练1/train_both_his2.txt'
train_combined_path = '训练1/train_combined_with_merchant_info_with_store_count.txt'
test_both_path = '测试1/test_both_his2.txt'
test_combined_path = '测试1/test_combined_with_merchant_info_with_store_count.txt'
clean_data_path = '流水数据/clean.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())

# 读取并检查文件中的 merchant_id
def check_merchant_ids(file_path):
    data = pd.read_csv(file_path)
    merchant_ids = set(data['merchant_id'].unique())
    total_ids = len(merchant_ids)
    missing_ids = merchant_ids - clean_merchant_ids
    missing_count = len(missing_ids)
    return total_ids, missing_count

# 检查每个文件
train_both_total, train_both_missing = check_merchant_ids(train_both_path)
train_combined_total, train_combined_missing = check_merchant_ids(train_combined_path)
test_both_total, test_both_missing = check_merchant_ids(test_both_path)
test_combined_total, test_combined_missing = check_merchant_ids(test_combined_path)

# 输出结果
print(f"train_both_his2.txt: Total Merchant IDs: {train_both_total}, Missing in clean.txt: {train_both_missing}")
print(f"train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {train_combined_total}, Missing in clean.txt: {train_combined_missing}")
print(f"test_both_his2.txt: Total Merchant IDs: {test_both_total}, Missing in clean.txt: {test_both_missing}")
print(f"test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {test_combined_total}, Missing in clean.txt: {test_combined_missing}")


train_both_his2.txt: Total Merchant IDs: 166, Missing in clean.txt: 0
train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 942, Missing in clean.txt: 0
test_both_his2.txt: Total Merchant IDs: 67, Missing in clean.txt: 0
test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 421, Missing in clean.txt: 0


In [23]:
import pandas as pd

# 文件路径
clean_data_path = '流水数据/clean.txt'
train_both_path = '训练1/train_both_his2.txt'
train_combined_path = '训练1/train_combined_with_merchant_info_with_store_count.txt'
test_both_path = '测试1/test_both_his2.txt'
test_combined_path = '测试1/test_combined_with_merchant_info_with_store_count.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())
print(f"流水数据 'clean.txt' 中不同的 merchant_id 数量: {len(clean_merchant_ids)}")

# 读取并汇总所有文件中的 merchant_id
files = [train_both_path, train_combined_path, test_both_path, test_combined_path]
all_merchant_ids = set()

for file in files:
    data = pd.read_csv(file)
    all_merchant_ids.update(data['merchant_id'].unique())

print(f"四个文件中不同的 merchant_id 总数: {len(all_merchant_ids)}")


流水数据 'clean.txt' 中不同的 merchant_id 数量: 1586
四个文件中不同的 merchant_id 总数: 1586


#### 1.2 发现在四个文件中相同merchant_id的贷款申请日期一定是一样的

In [30]:
import pandas as pd

# 定义文件路径列表
file_paths = [
    '训练1/train_both_his2.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_both_his2.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt'
]

# 合并所有文件中的 merchant_id 和 apply_date
all_data = pd.DataFrame()
for path in file_paths:
    data = pd.read_csv(path, usecols=['merchant_id', 'apply_date'])
    all_data = pd.concat([all_data, data], ignore_index=True)

# 转换 apply_date 到 datetime 类型
all_data['apply_date'] = pd.to_datetime(all_data['apply_date'], format='%Y%m%d', errors='coerce')

# 分组 by merchant_id, 检查是否有多个不同的 apply_date
grouped = all_data.groupby('merchant_id')['apply_date'].nunique()

# 找到有多个不同 apply_date 的 merchant_id
multiple_dates = grouped[grouped > 1]

if not multiple_dates.empty:
    print(f"存在 {len(multiple_dates)} 个 merchant_id 有多个不同的 apply_date。")
    print("其中一些包括：")
    print(multiple_dates.head())  # 显示一些示例
else:
    print("所有 merchant_id 的 apply_date 都相同。")


所有 merchant_id 的 apply_date 都相同。


#### 1.3 以某一个merchant_id为例，发现流水记录中存在申请日期后的交易记录

In [40]:
import pandas as pd
from datetime import timedelta

# 加载训练数据，以确保我们得到正确的 apply_date
file_path = '训练1/train_both_his2.txt'
data = pd.read_csv(file_path)
data['apply_date'] = pd.to_datetime(data['apply_date'])

# 定义 merchant_id
merchant_id_to_find = 934378057859389

# 找到指定 merchant_id 的 apply_date
merchant_apply_date = data.loc[data['merchant_id'] == merchant_id_to_find, 'apply_date'].max()

# 加载流水数据
clean_data = pd.read_csv('流水数据/clean.txt')
clean_data['pt'] = pd.to_datetime(clean_data['pt'], format='%Y%m%d')

# 计算申请日期的前半年范围
start_date = merchant_apply_date - timedelta(days=180)

# 筛选流水记录
merchant_records = clean_data[clean_data['merchant_id'] == merchant_id_to_find]
valid_records = merchant_records[(merchant_records['pt'] >= start_date) & (merchant_records['pt'] <= merchant_apply_date)]

print(f"merchant_id: {merchant_id_to_find}")
print(f"最晚申请日期: {merchant_apply_date}")
print(f"有效日期开始: {start_date}")
print(f"符合条件的流水记录数: {len(valid_records)}")
print(f"总流水记录数: {len(merchant_records)}")
print("符合条件的记录详情:")
print(valid_records[['pt', 'paid_amount']])  # 简化输出，仅显示日期和支付金额


merchant_id: 934378057859389
最晚申请日期: 2018-06-08 00:00:00
有效日期开始: 2017-12-10 00:00:00
符合条件的流水记录数: 3628
总流水记录数: 4031
符合条件的记录详情:
                pt  paid_amount
657     2018-04-22         5200
1707    2018-04-22         2100
2964    2018-04-22         2800
3040    2018-04-22         2200
3695    2018-04-22         1300
...            ...          ...
3418933 2018-04-14         1300
3420574 2018-04-17         1300
3421592 2018-04-18         1100
3421750 2018-04-18         1700
3421859 2018-04-18         1600

[3628 rows x 2 columns]


In [39]:
import pandas as pd

# 定义 merchant_id
merchant_id_to_find = 934378057859389

# 定义文件路径列表
file_paths = [
    '训练1/train_both_his2.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_both_his2.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt'
]

# 循环遍历文件列表，查找特定 merchant_id 的记录
for path in file_paths:
    data = pd.read_csv(path)
    # 筛选出特定 merchant_id 的记录
    filtered_data = data[data['merchant_id'] == merchant_id_to_find]
    if not filtered_data.empty:
        print(f"在文件 {path} 中找到的记录:")
        print(filtered_data)
    else:
        print(f"在文件 {path} 中未找到 merchant_id 为 {merchant_id_to_find} 的记录。")

# 加载流水数据
clean_data = pd.read_csv('流水数据/clean.txt')

# 在流水数据中查找特定 merchant_id 的记录
clean_filtered = clean_data[clean_data['merchant_id'] == merchant_id_to_find]
if not clean_filtered.empty:
    print("在流水数据 clean.txt 中找到的记录:")
    print(clean_filtered)
    # 输出该 merchant_id 对应的 pt 的最小和最大日期
    pt_min = clean_filtered['pt'].min()
    pt_max = clean_filtered['pt'].max()
    print(f"该 merchant_id {merchant_id_to_find} 的流水记录日期范围：从 {pt_min} 到 {pt_max}")
else:
    print(f"在流水数据 clean.txt 中未找到 merchant_id 为 {merchant_id_to_find} 的记录。")


在文件 训练1/train_both_his2.txt 中找到的记录:
   apply_id      merchant_id           apply_date  lend_amount  lend_period  \
0  52849963  934378057859389  2018-06-08 00:00:00      50000.0            3   

   is_30days_overdue  industry_level1  province  store_count  \
0                  0                1         2            1   

   his_loan_amount_all  avg_period  loan_count  overdue_count  overdue_rate  \
0              43000.0        12.0           1              0           0.0   

   overdue_days_all  overdue_sum  
0                 0            0  
在文件 训练1/train_combined_with_merchant_info_with_store_count.txt 中未找到 merchant_id 为 934378057859389 的记录。
在文件 测试1/test_both_his2.txt 中未找到 merchant_id 为 934378057859389 的记录。
在文件 测试1/test_combined_with_merchant_info_with_store_count.txt 中未找到 merchant_id 为 934378057859389 的记录。
在流水数据 clean.txt 中找到的记录:
                        id         store_id      merchant_id  pay_way  \
657      t7895258640472291  863886432460409  934378057859389        3   
1707 

#### 1.4 根据每个merchant_id的贷款申请日期，只保留之前的流水记录

In [54]:
import pandas as pd
from datetime import timedelta

# 定义文件路径列表
file_paths = [
    '训练1/train_both_his2.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_both_his2.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt'
]

# 合并所有文件中的 merchant_id 和 apply_date
all_data = pd.DataFrame()
for path in file_paths:
    data = pd.read_csv(path, usecols=['merchant_id', 'apply_date'])
    all_data = pd.concat([all_data, data], ignore_index=True)

# 去除重复项，并转换 apply_date 为 datetime 类型
all_data['apply_date'] = pd.to_datetime(all_data['apply_date'], errors='coerce')
unique_data = all_data.drop_duplicates()

# 加载流水数据并转换日期格式
clean_data = pd.read_csv('流水数据/clean.txt')
clean_data['pt'] = pd.to_datetime(clean_data['pt'], format='%Y%m%d')

# 准备筛选后的数据容器
filtered_clean_data = pd.DataFrame()

# 统计删除的记录数
deleted_records = 0

# 筛选出 pt 日期早于 apply_date 半年之内的记录
for _, row in unique_data.iterrows():
    merchant_id = row['merchant_id']
    apply_date = row['apply_date']
    start_date = apply_date - timedelta(days=180)  # 计算半年前的日期
    
    # 获取该 merchant_id 的所有流水记录
    merchant_records = clean_data[clean_data['merchant_id'] == merchant_id]
    
    # 筛选出 pt 日期在半年之内的记录
    valid_records = merchant_records[(merchant_records['pt'] >= start_date) & (merchant_records['pt'] < apply_date)]
    filtered_clean_data = pd.concat([filtered_clean_data, valid_records], ignore_index=True)
    
    # 计算删除的记录数
    deleted_records += (len(merchant_records) - len(valid_records))

# 保存筛选后的数据到新文件 clean1.txt
filtered_clean_data.to_csv('流水数据/clean1.txt', index=False)

# 输出必要的数据规模信息
original_size = len(clean_data)
new_size = len(filtered_clean_data)
print(f"原始流水数据记录数：{original_size}")
print(f"保留的流水数据记录数：{new_size}")
print(f"删除的流水记录数：{deleted_records}")


原始流水数据记录数：3422391
保留的流水数据记录数：2210268
删除的流水记录数：1290793


In [56]:
import pandas as pd

# 文件路径
train_both_path = '训练1/train_both_his2.txt'
train_combined_path = '训练1/train_combined_with_merchant_info_with_store_count.txt'
test_both_path = '测试1/test_both_his2.txt'
test_combined_path = '测试1/test_combined_with_merchant_info_with_store_count.txt'
clean_data_path = '流水数据/clean1.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())

# 读取并检查文件中的 merchant_id
def check_merchant_ids(file_path):
    data = pd.read_csv(file_path)
    merchant_ids = set(data['merchant_id'].unique())
    total_ids = len(merchant_ids)
    missing_ids = merchant_ids - clean_merchant_ids
    missing_count = len(missing_ids)
    return total_ids, missing_count

# 检查每个文件
train_both_total, train_both_missing = check_merchant_ids(train_both_path)
train_combined_total, train_combined_missing = check_merchant_ids(train_combined_path)
test_both_total, test_both_missing = check_merchant_ids(test_both_path)
test_combined_total, test_combined_missing = check_merchant_ids(test_combined_path)

# 输出结果
print(f"train_both_his2.txt: Total Merchant IDs: {train_both_total}, Missing in clean.txt: {train_both_missing}")
print(f"train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {train_combined_total}, Missing in clean.txt: {train_combined_missing}")
print(f"test_both_his2.txt: Total Merchant IDs: {test_both_total}, Missing in clean.txt: {test_both_missing}")
print(f"test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: {test_combined_total}, Missing in clean.txt: {test_combined_missing}")


train_both_his2.txt: Total Merchant IDs: 163, Missing in clean.txt: 0
train_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 893, Missing in clean.txt: 0
test_both_his2.txt: Total Merchant IDs: 65, Missing in clean.txt: 0
test_combined_with_merchant_info_with_store_count.txt: Total Merchant IDs: 396, Missing in clean.txt: 0


In [57]:
import pandas as pd

# 文件路径
clean_data_path = '流水数据/clean1.txt'
train_both_path = '训练1/train_both_his2.txt'
train_combined_path = '训练1/train_combined_with_merchant_info_with_store_count.txt'
test_both_path = '测试1/test_both_his2.txt'
test_combined_path = '测试1/test_combined_with_merchant_info_with_store_count.txt'

# 读取流水数据中的 merchant_id
clean_data = pd.read_csv(clean_data_path)
clean_merchant_ids = set(clean_data['merchant_id'].unique())
print(f"流水数据 'clean1.txt' 中不同的 merchant_id 数量: {len(clean_merchant_ids)}")

# 读取并汇总所有文件中的 merchant_id
files = [train_both_path, train_combined_path, test_both_path, test_combined_path]
all_merchant_ids = set()

for file in files:
    data = pd.read_csv(file)
    all_merchant_ids.update(data['merchant_id'].unique())

print(f"四个文件中不同的 merchant_id 总数: {len(all_merchant_ids)}")


流水数据 'clean.txt' 中不同的 merchant_id 数量: 1507
四个文件中不同的 merchant_id 总数: 1507


### 2、 对经过筛选的流水记录增加avg_discount,avg_paid,avg_order,avg_balance,avg_advance五个字段

#####   avg_balance: 商户储蓄卡、微信余额、支付宝余额、支付宝余额宝的总支付金额占总支付金额的比例（不预支组）

#####  商户信用卡、花呗、集分宝的总支付金额占总支付金额的比例（预支组）

In [73]:
import pandas as pd

# 加载数据
data_path = '流水数据/clean1.txt'
data = pd.read_csv(data_path)

# 转换 pt 字段为日期格式（假设 pt 是格式如 '2018-04-22' 的日期表示）
data['date'] = pd.to_datetime(data['pt'], format='%Y-%m-%d')

# 定义涉及到的不同字段组
payment_columns = [
    'bankcard_credit', 'bankcard_debit', 'wallet_weixin',
    'wallet_alipay', 'wallet_alipay_finance', 'alipay_huabei', 'alipay_point'
]
balance_fields = ['bankcard_debit', 'wallet_weixin', 'wallet_alipay', 'wallet_alipay_finance']
advance_fields = ['bankcard_credit', 'alipay_huabei', 'alipay_point']

# 确保所有涉及运算的字段都是数值类型
data[payment_columns + ['paid_amount']] = data[payment_columns + ['paid_amount']].apply(pd.to_numeric, errors='coerce')

# 计算每个订单的实际支付总和并检测支付不足的订单
data['real_sum'] = data[payment_columns].sum(axis=1)
underpaid_count = (data['real_sum'] < data['paid_amount']).sum()
data['discount'] = data.apply(lambda x: x['real_sum'] - x['paid_amount'] if x['real_sum'] >= x['paid_amount'] else 0, axis=1)

# 计算订单统计信息
daily_order_counts = data.groupby(['merchant_id', 'date'])['id'].nunique().reset_index(name='order_count')
avg_order_per_merchant = daily_order_counts.groupby('merchant_id')['order_count'].mean().reset_index(name='avg_order')

# 计算余额和预付信息
data['pay_balance'] = data[balance_fields].sum(axis=1)
data['balance_rate'] = data['pay_balance'] / data['paid_amount']
data['pay_advance'] = data[advance_fields].sum(axis=1)
data['advance_rate'] = data['pay_advance'] / data['paid_amount']

# 分组计算每个merchant_id的各类统计信息
grouped_discount = data.groupby(['merchant_id', 'date'])['discount'].sum().reset_index()
avg_discount = grouped_discount.groupby('merchant_id')['discount'].mean().reset_index(name='avg_discount')
grouped_data = data.groupby('merchant_id').agg(avg_balance=('balance_rate', 'mean'), avg_advance=('advance_rate', 'mean')).reset_index()

# 保存结果到新文件
avg_paid = data.groupby(['merchant_id', 'date'])['paid_amount'].sum().reset_index(name='paid_amount_sum').groupby('merchant_id')['paid_amount_sum'].mean().reset_index(name='avg_paid')
avg_paid.to_csv('流水数据/avg_paid.txt', index=False)
avg_discount.to_csv('流水数据/avg_discount.txt', index=False)
grouped_data.to_csv('流水数据/rate.txt', index=False, sep=',')
avg_order_per_merchant.to_csv('流水数据/order.txt', index=False, sep=',')

# 输出
print(f"文件已保存，包含每个商户的平均每日支付金额、红包金额、订单数、balance_rate和advance_rate。")
print(f"实际支付总和小于已支付金额的订单个数: {underpaid_count}")


文件已保存，包含每个商户的平均每日支付金额、红包金额、订单数、balance_rate和advance_rate。
实际支付总和小于已支付金额的订单个数: 7520


#### 2.1、 形成一个只有merchant_id、avg_discount、avg_paid、avg_order、avg_balance、avg_advance六个字段的文件

In [74]:
import pandas as pd

# 加载数据
avg_paid = pd.read_csv('流水数据/avg_paid.txt')
avg_discount = pd.read_csv('流水数据/avg_discount.txt')
rate = pd.read_csv('流水数据/rate.txt')
order = pd.read_csv('流水数据/order.txt')

# 打印原始 avg_paid 的维度 (行数, 列数)
original_shape = avg_paid.shape
print(f"原始 avg_paid.txt 文件的维度 (行数, 列数): {original_shape}")

# 合并 avg_discount
avg_paid = avg_paid.merge(avg_discount, on='merchant_id', how='left')

# 合并 rate，包含 avg_balance 和 avg_advance
avg_paid = avg_paid.merge(rate, on='merchant_id', how='left')

# 合并 order，包含 avg_order
avg_paid = avg_paid.merge(order, on='merchant_id', how='left')

# 打印合并后 avg_paid 的维度 (行数, 列数)
merged_shape = avg_paid.shape
print(f"合并后 avg_paid.txt 文件的维度 (行数, 列数): {merged_shape}")

# 保存合并后的数据到新文件，或覆盖原文件
avg_paid.to_csv('流水数据/avg_paid_updated.txt', index=False)

print("数据已更新并保存到 avg_paid_updated.txt。")


原始 avg_paid.txt 文件的维度 (行数, 列数): (1507, 2)
合并后 avg_paid.txt 文件的维度 (行数, 列数): (1507, 6)
数据已更新并保存到 avg_paid_updated.txt。


#### 2.2、 根据merchant_id的对应关系将来自流水数据的五个新特征合并到训练和测试文件中

In [75]:
import pandas as pd

# 定义文件路径列表
file_paths = [
    '训练1/train_both_his2.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_both_his2.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt'
]

# 加载 avg_paid_updated.txt
avg_paid_updated = pd.read_csv('流水数据/avg_paid_updated.txt')
avg_paid_ids = set(avg_paid_updated['merchant_id'])

# 遍历每个文件并检查其中的 merchant_id 是否都在 avg_paid_updated.txt 中
for path in file_paths:
    data = pd.read_csv(path)
    missing_ids = set(data['merchant_id']) - avg_paid_ids
    if missing_ids:
        print(f"在文件 {path} 中，以下 merchant_id 在 avg_paid_updated.txt 中未找到:", missing_ids)
    else:
        print(f"文件 {path} 中的所有 merchant_id 都在 avg_paid_updated.txt 中找到了。")


文件 训练1/train_both_his2.txt 中的所有 merchant_id 都在 avg_paid_updated.txt 中找到了。
文件 训练1/train_combined_with_merchant_info_with_store_count.txt 中的所有 merchant_id 都在 avg_paid_updated.txt 中找到了。
文件 测试1/test_both_his2.txt 中的所有 merchant_id 都在 avg_paid_updated.txt 中找到了。
文件 测试1/test_combined_with_merchant_info_with_store_count.txt 中的所有 merchant_id 都在 avg_paid_updated.txt 中找到了。


In [76]:
import pandas as pd

# 加载 avg_paid_updated 数据
avg_paid_updated = pd.read_csv('流水数据/avg_paid_updated.txt')

# 定义文件路径列表
file_paths = [
    '训练1/train_both_his2.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_both_his2.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt'
]

# 对每个文件进行处理
for path in file_paths:
    # 加载当前文件
    data = pd.read_csv(path)
    print(f"原始 {path} 文件的维度 (行数, 列数): {data.shape}")

    # 合并 avg_paid 更新的信息
    merged_data = pd.merge(data, avg_paid_updated, on='merchant_id', how='left')

    # 打印合并后的文件规模
    print(f"合并后的 {path} 文件的维度 (行数, 列数): {merged_data.shape}")

    # 保存新的合并文件
    new_path = path.replace('.txt', '_updated.txt')
    merged_data.to_csv(new_path, index=False)
    print(f"更新后的数据已保存到 {new_path}\n")


原始 训练1/train_both_his2.txt 文件的维度 (行数, 列数): (164, 16)
合并后的 训练1/train_both_his2.txt 文件的维度 (行数, 列数): (164, 21)
更新后的数据已保存到 训练1/train_both_his2_updated.txt

原始 训练1/train_combined_with_merchant_info_with_store_count.txt 文件的维度 (行数, 列数): (910, 9)
合并后的 训练1/train_combined_with_merchant_info_with_store_count.txt 文件的维度 (行数, 列数): (910, 14)
更新后的数据已保存到 训练1/train_combined_with_merchant_info_with_store_count_updated.txt

原始 测试1/test_both_his2.txt 文件的维度 (行数, 列数): (65, 15)
合并后的 测试1/test_both_his2.txt 文件的维度 (行数, 列数): (65, 20)
更新后的数据已保存到 测试1/test_both_his2_updated.txt

原始 测试1/test_combined_with_merchant_info_with_store_count.txt 文件的维度 (行数, 列数): (400, 8)
合并后的 测试1/test_combined_with_merchant_info_with_store_count.txt 文件的维度 (行数, 列数): (400, 13)
更新后的数据已保存到 测试1/test_combined_with_merchant_info_with_store_count_updated.txt

